In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import zipfile
from sklearn.metrics import mean_squared_error

c:\Users\Matias\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\Matias\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
zip_path = '../00_Data_Bases/df_recommendationsystem.zip' 
csv_filename = 'df_recommendationsystem.csv'

with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open(csv_filename) as f:
        df_cl4 = pd.read_csv(f)

In [3]:
df_cl4.head()

,user_id,department_babies,department_bakery,department_beverages,department_breakfast,department_bulk,department_canned goods,department_dairy eggs,department_deli,department_dry goods pasta,...,department_missing,department_other,department_pantry,department_personal care,department_pets,department_produce,department_snacks,max_order,Clusters_4,Customer_Type
0,147243,0,1,1,2,0,2,31,7,3,...,0,0,4,1,0,18,4,38,2,Básicos Frecuentes
1,195275,0,3,4,0,0,0,6,1,4,...,0,0,1,0,0,24,5,90,0,Rutina Estable
2,58222,0,0,1,0,0,0,6,0,0,...,0,0,4,0,0,3,0,3,1,Diversificados
3,120094,0,0,2,0,0,0,1,0,0,...,0,0,1,0,0,3,1,18,1,Diversificados
4,53959,2,0,3,1,0,0,4,0,0,...,0,0,0,9,0,5,2,8,3,Grandes Compras


In [4]:
zip_path = '../00_Data_Bases/Supermercado_onehot.zip' 
csv_filename = 'Supermercado_onehot.csv'

with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open(csv_filename) as f:
        df = pd.read_csv(f)

In [5]:
df_rs = df[['user_id','max_order', 
                 'department_babies', 'department_bakery', 
                 'department_beverages', 'department_breakfast', 'department_bulk', 
                 'department_canned goods', 'department_dairy eggs', 'department_deli', 
                 'department_dry goods pasta', 'department_frozen', 'department_household', 
                 'department_international', 'department_meat seafood', 'department_missing', 
                 'department_other', 'department_pantry', 'department_personal care', 
                 'department_pets', 'department_produce', 'department_snacks', 
                 'product_name_asian foods', 'product_name_baby accessories', 
                 'product_name_baby bath body care', 'product_name_baby food formula', 
                 'product_name_bakery desserts', 'product_name_baking ingredients', 
                 'product_name_baking supplies decor', 'product_name_beauty']]


In [6]:
df_rs.head()

,user_id,max_order,department_babies,department_bakery,department_beverages,department_breakfast,department_bulk,department_canned goods,department_dairy eggs,department_deli,...,department_produce,department_snacks,product_name_asian foods,product_name_baby accessories,product_name_baby bath body care,product_name_baby food formula,product_name_bakery desserts,product_name_baking ingredients,product_name_baking supplies decor,product_name_beauty
0,147243,38,0,1,1,2,0,2,31,7,...,18,4,0,0,0,0,1,0,0,0
1,195275,90,0,3,4,0,0,0,6,1,...,24,5,0,0,0,0,0,0,0,0
2,58222,3,0,0,1,0,0,0,6,0,...,3,0,0,0,0,0,0,3,0,0
3,120094,18,0,0,2,0,0,0,1,0,...,3,1,0,0,0,0,0,0,0,0
4,53959,8,2,0,3,1,0,0,4,0,...,5,2,0,0,0,0,0,0,0,0


In [7]:
# Realizamos la combinación de los datasets
df_rs = df_rs.merge(
    df_cl4[['user_id', 'Clusters_4', 'Customer_Type']],  # Seleccionamos las columnas necesarias
    on='user_id',  # Columna en común
    how='left'  # Usamos 'left' para mantener todas las filas de df_rs
)

# Verificamos las primeras filas del dataset combinado
df_rs.head()


,user_id,max_order,department_babies,department_bakery,department_beverages,department_breakfast,department_bulk,department_canned goods,department_dairy eggs,department_deli,...,product_name_asian foods,product_name_baby accessories,product_name_baby bath body care,product_name_baby food formula,product_name_bakery desserts,product_name_baking ingredients,product_name_baking supplies decor,product_name_beauty,Clusters_4,Customer_Type
0,147243,38,0,1,1,2,0,2,31,7,...,0,0,0,0,1,0,0,0,2,Básicos Frecuentes
1,195275,90,0,3,4,0,0,0,6,1,...,0,0,0,0,0,0,0,0,0,Rutina Estable
2,58222,3,0,0,1,0,0,0,6,0,...,0,0,0,0,0,3,0,0,1,Diversificados
3,120094,18,0,0,2,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,Diversificados
4,53959,8,2,0,3,1,0,0,4,0,...,0,0,0,0,0,0,0,0,3,Grandes Compras
